In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import sys, os

In [2]:
#adding scripts path
sys.path.append(os.path.abspath(os.path.join('..')))
#import scripts
from scripts.data_extractor import DataExtractor
from scripts.data_manipulator import DataManipulator

In [3]:
#instantiate objects
extractor = DataExtractor()
manipulator = DataManipulator()

### Rain vs No-Rain

### Holiday vs Not-Holiday

### Traffic Condition

### Weekday vs Weekend